<a href="https://colab.research.google.com/github/Renga1610/Renga1610/blob/main/Lyrics_Video_Generation_Using_Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The idea to create lyrics using Video Generation for song lyrics, using MusixMatch Lyrics API.

The api gives the song lyrics only and not the song itself or their timestamps. 



So, it strictly gives back GIF created from the lyrics with constant timestamp.


In [ ]:
import requests
import json

In [ ]:
!pip install keras-cv --upgrade --quiet

In [ ]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from PIL import Image, ImageFont, ImageDraw

# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(jit_compile=True)

Input of the Artist and Song Name.
It will fetch lyrics from musixmatch api

In [ ]:
artist_name = "the weeknd"
track_name = "blinding lights"

In [ ]:
base_url = "https://api.musixmatch.com/ws/1.1/"

# your api key
api_key = "&apikey=a8184334ca9d25054d1022b742243663"
lyrics_matcher = "matcher.lyrics.get"
format_url = "?format=json&callback=callback"
artist_search_parameter = "&q_artist="
track_search_parameter = "&q_track="

api_call = base_url + lyrics_matcher + format_url + artist_search_parameter + artist_name + track_search_parameter + track_name + api_key

request = requests.get(api_call)
data = request.json()
data = data['message']['body']
print("API Call: " + api_call)
print()
lyrics = data['lyrics']['lyrics_body']
print(data['lyrics']['lyrics_body'])

In [ ]:
lyric_list = lyrics.split("\n")

Lyrics of the song

In [ ]:
lyric_list

I have given interpolation steps as 3 (constant).

Timestamps data can be used to make a proper lyric format in future

In [ ]:
interpolation_steps = 3
encoding = []

for i in range(len(lyric_list)):
  encoding.append(tf.squeeze(model.encode_text(lyric_list[i])))

interpolated_encodings = []

for i in range(len(lyric_list)-1):
  interpolated_encodings.append(tf.linspace(encoding[i], encoding[i+1], interpolation_steps))

# Show the size of the latent manifold
print(f"Encoding shape: {encoding[i].shape}")

In [ ]:
len(encoding)

In [ ]:
seed = 12345
noise = tf.random.normal((512 // 8, 512 // 8, 4), seed=seed)
images = []
for i in range(len(interpolated_encodings)):
  images.append(model.generate_image(
    interpolated_encodings[i],
    batch_size=interpolation_steps,
    diffusion_noise=noise,
))

In [ ]:
import numpy as np
images = np.array(images)

In [ ]:
images = images.reshape(-1,512,512,3)

In [ ]:
pil_image = [Image.fromarray(img) for img in images]
pil_d_image = []
j = 0
for i in range(len(pil_image)):
  draw = ImageDraw.Draw(pil_image[i])
  # use a bitmap font
  font = ImageFont.load_default()
  if i ==0:
    lyric_draw = lyric_list[0]
  elif i%3==0:
    lyric_draw = lyric_list[j+1]
    j = j+1
  else:
    lyric_draw = lyric_list[j]
  draw.text((100, 100), lyric_draw, font=font, fill ="red")
  pil_d_image.append(pil_image[i])

In [ ]:
def export_as_gif(filename, images, frames_per_second=10, rubber_band=False):
    if rubber_band:
        images += images[2:-1][::-1]

    images[0].save(
        filename,
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )


export_as_gif(
    "lyric_video.gif",
    pil_d_image,
    frames_per_second=2,
    rubber_band=True,
)

In [ ]:
from IPython.display import Image as IImage
IImage("lyric_video.gif")